In [3]:
# 1. Install necessary libraries
# We use 'quiet' to keep the output clean, remove it if you want to see installation details
!pip install pandas nltk textblob --quiet

# 2. Import modules
import pandas as pd
import random
import json
from datetime import datetime
import nltk
from textblob import TextBlob # Good for simple sentiment detection (keeping mood positive)

# 3. Download required NLTK data for tokenizing text
nltk.download('punkt', quiet=True)
nltk.download('brown', quiet=True) # Useful corpus for reference

# 4. Define a simple configuration for the bot
BOT_CONFIG = {
    "bot_name": "Remi", # Short for Reminiscence
    "user_name": "User", # We can change this dynamically later
    "session_start": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

print(f"✅ Environment Ready for {BOT_CONFIG['bot_name']}!")
print(f"Session started at: {BOT_CONFIG['session_start']}")

✅ Environment Ready for Remi!
Session started at: 2026-02-09 22:27:28


From the code above, 

pandas: We will use this to save conversation logs (very important for analysis later).

textblob: A simple library to detect sentiment. If a user gets agitated (negative sentiment), we can program the bot to change the subject.

BOT_CONFIG: A dictionary to store basic settings we can reference later.

We now want to move to Step 2: Defining the "Reminiscence Knowledge Base" This is the brain the bot will draw from.

In [4]:
# Define the topics and specific reminiscence prompts
reminiscence_topics = {
    "childhood": [
        "What was your favorite game to play outside when you were young?",
        "Do you remember the house you grew up in? What was your favorite room?",
        "What was your favorite home-cooked meal as a child?",
        "Did you have any pets growing up? What were their names?"
    ],
    "career": [
        "What was your very first job? Did you enjoy it?",
        "What is a professional accomplishment you are most proud of?",
        "Who was the best boss or mentor you ever had?",
        "How did your work change over the years?"
    ],
    "hobbies": [
        "What hobbies have you enjoyed throughout your life?",
        "Is there a particular song or type of music that always makes you smile?",
        "Did you enjoy traveling? Where was your favorite place to visit?",
        "What is a book or movie that had a big impact on you?"
    ],
    "family": [
        "How did you meet your spouse or partner?",
        "What is a favorite holiday tradition your family had?",
        "Tell me about a memorable family vacation.",
        "What values were most important to your parents?"
    ]
}

# Simple function to get a random question from a topic
def get_prompt(topic):
    if topic in reminiscence_topics:
        return random.choice(reminiscence_topics[topic])
    else:
        return "I'd love to hear more about that. Can you tell me more?"

print("✅ Knowledge Base Loaded with topics:", list(reminiscence_topics.keys()))

✅ Knowledge Base Loaded with topics: ['childhood', 'career', 'hobbies', 'family']


We are keeping the conversation focused on therapeutic areas and later we can add other information like music, school, e.t.c.

Defining The Logic Functions:

Next, we are going to build the "Brain" It is a function that takes what the user says, analyzes it and decides the memory topic to trigger next.

In [5]:
# --- REMI V6.1: Softer Empathy Layer ---

import random
from textblob import TextBlob

# Emotion-aware response pools (less clinical, more human)
empathy_responses = {
    "bad": [
        "I'm sorry today feels bad.",
        "That sounds like a hard day.",
        "Some days are just heavy."
    ],
    "sad": [
        "That sounds really sad.",
        "I'm here with you.",
        "I'm glad you told me."
    ],
    "tired": [
        "Feeling tired can be exhausting.",
        "Maybe your body needs some rest.",
        "We can take things slowly."
    ],
    "lonely": [
        "Feeling lonely is really hard.",
        "You're not alone right now.",
        "I'm here with you."
    ],
    "angry": [
        "That sounds frustrating.",
        "It's okay to feel upset.",
        "I hear how strong that feeling is."
    ],
    "anxious": [
        "It's okay. You're safe right now.",
        "Let's slow down together.",
        "Nothing is urgent."
    ]
}

grounding_phrases = [
    "You're safe right now.",
    "Nothing needs to be done.",
    "We can just sit for a moment."
]

chat_state = {
    "current_topic": None,
    "asked_questions": set(),
    "standing_by": False,
    "comfort_mode": False
}

print("✅ Remi V6.1 Empathy Layer Loaded")


✅ Remi V6.1 Empathy Layer Loaded


In [ ]:
# --- REMI V8: The Companion (with Memory) ---

import random
import datetime
from textblob import TextBlob
import re # We need "Regular Expressions" to find names/facts

# 1. THE BRAIN (Now with Long-Term Memory)
chat_state = {
    "standing_by": False,
    "user_profile": {
        "name": None,
        "likes": [],
        "mood": "neutral"
    }
}

# 2. TIME AWARENESS FUNCTION
def get_time_greeting():
    hour = datetime.datetime.now().hour
    if hour < 12:
        return "Good morning"
    elif 12 <= hour < 18:
        return "Good afternoon"
    else:
        return "Good evening"

# 3. MEMORY FUNCTIONS
def extract_information(text):
    text = text.lower()
    
    # Try to find a name using pattern matching
    # Looks for "i am [name]" or "my name is [name]"
    name_match = re.search(r"(?:i am|my name is|call me) (\w+)", text)
    if name_match:
        # Save the name to memory!
        captured_name = name_match.group(1).capitalize()
        # Avoid saving "hungry" or "sad" as a name
        if captured_name not in ["hungry", "sad", "tired", "happy", "bad", "safe"]:
            chat_state["user_profile"]["name"] = captured_name
            return f"I will remember that. It is nice to meet you, {captured_name}."
            
    return None

def get_smart_response(user_input):
    text = user_input.lower().strip()
    
    # --- LAYER 1: CHECK MEMORY FIRST ---
    # Run the learning function to see if the user shared a fact
    memory_reply = extract_information(user_input)
    if memory_reply:
        return memory_reply

    # --- LAYER 2: STANDBY & SHUTDOWN ---
    if chat_state["standing_by"]:
        # Only wake up if we hear the specific name (if we know it) or "Remi"
        trigger_words = ["remi", "hello", "wake up"]
        if chat_state["user_profile"]["name"]:
            trigger_words.append(chat_state["user_profile"]["name"].lower())
            
        if any(w in text for w in trigger_words):
            chat_state["standing_by"] = False
            user_name = chat_state["user_profile"]["name"] or "my friend"
            return f"I am back. I am listening, {user_name}."
        return "..."

    if any(w in text for w in ["shut up", "go away", "stop", "leave me"]):
        chat_state["standing_by"] = True
        return "I will be quiet now. I am here if you need me."

    # --- LAYER 3: PERSONALIZED GREETINGS ---
    if any(w in text for w in ["hello", "hi"]):
        greeting = get_time_greeting()
        user_name = chat_state["user_profile"]["name"]
        
        if user_name:
            return f"{greeting}, {user_name}. It is good to see you again."
        else:
            return f"{greeting}. I don't think I know your name yet. What should I call you?"

    # --- LAYER 4: CONTEXTUAL RESPONSES ---
    if "hungry" in text:
        return "You should eat. Do you want me to pause while you get food?"
    
    if "sad" in text or "bad" in text:
        user_name = chat_state["user_profile"]["name"]
        if user_name:
             return f"I am sorry you are feeling this way, {user_name}. I am here with you."
        return "I am sorry today is hard. I am here with you."

    # --- LAYER 5: FALLBACK (Use Memory) ---
    # If we don't know what to say, use the name to make it sound personal
    user_name = chat_state["user_profile"]["name"]
    if user_name:
        return f"I am listening, {user_name}. Tell me more."
    else:
        return "I am listening. Tell me more."

# -----------------------------
# INTERACTIVE LOOP
# -----------------------------
print(f"--- Remi V8 (The Companion) ---")
print("Remi: I am online.")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit"]:
        print("Remi: Goodbye.")
        break
        
    response = get_smart_response(user_input)
    print(f"Remi: {response}")

--- Remi V8 (The Companion) ---
Remi: I am online.


You:  Hello there


Remi: Good evening. I don't think I know your name yet. What should I call you?


You:  I don't remember my name


Remi: I am listening. Tell me more.


...

Remi: I am listening. Tell me more.


You:  are you ok?


Remi: I am listening. Tell me more.


You:  stop answering those vague things


Remi: I will be quiet now. I am here if you need me.


The above is important because of context awareness, i.e if the user says I loved my mum, the bot will detect family and ask a family question next. It acts as a safety valve i.e < -0.5 check if the patient gets upset, the bot tries to pivot to a happier subject(music/hobbies)

we are now going to stitch all those pieces together into a running loop. This will be the main chat loop.

We now want to introduce long term memory to our bot